# 콘텐츠 기반 필터링  

# 협업 필터링
- 최근접 이웃 기반 협업 필터링  
    - 사용자 기반
    - 아이템 기반
- 잠재적 요인 협업 필터링 - 행렬 변환

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [80]:
movies = pd.read_csv('./tmdb_5000_movies.csv')
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [81]:
movies.shape

(4803, 20)

In [82]:
movies_df = movies[['id', 'title','genres','vote_count','vote_average','popularity','keywords','overview']]

In [83]:
movies_df

,id,title,genres,vote_count,vote_average,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",11800,7.2,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",4500,6.9,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",4466,6.3,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",9106,7.6,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2124,6.1,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",238,6.6,14.269792,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...
4799,72766,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",5,5.9,0.642552,[],A newlywed couple's honeymoon is upended by th...
4800,231617,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",6,7.0,1.444476,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,126186,Shanghai Calling,[],7,5.7,0.857008,[],When ambitious New York attorney Sam is sent t...


# 콘텐츠 기반 필터링 추천 시스템

In [84]:
movies_df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [85]:
movies_df['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [86]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_count    4803 non-null   int64  
 4   vote_average  4803 non-null   float64
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


In [87]:
# str로 된 python code를 실제 python code로 변환 : eval, literal_eval
type(movies_df['genres'][0])

str

In [88]:
from ast import literal_eval

type(literal_eval(movies_df['genres'][0]))

list

In [89]:
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_1284/4131196216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(literal_eval)
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_1284/4131196216.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)


In [90]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_count    4803 non-null   int64  
 4   vote_average  4803 non-null   float64
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


In [91]:
movies_df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [92]:
[d['name'] for d in movies_df['genres'][0]]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [93]:
[d['name'] for d in movies_df['keywords'][0]]

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [94]:
movies_df['genres'] = movies_df['genres'].apply(lambda x:[d['name'] for d in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[d['name'] for d in x])


C:\Users\bitcamp\AppData\Local\Temp/ipykernel_1284/3654860830.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(lambda x:[d['name'] for d in x])
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_1284/3654860830.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[d['name'] for d in x])


In [95]:
movies_df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [96]:
movies_df.head()

,id,title,genres,vote_count,vote_average,popularity,keywords,overview
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",11800,7.2,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",4500,6.9,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[Action, Adventure, Crime]",4466,6.3,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",9106,7.6,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...
4,49529,John Carter,"[Action, Adventure, Science Fiction]",2124,6.1,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca..."


In [97]:
# CountVector ilzer에 넣기 위해서 문자열 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_1284/3495623500.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))


In [ ]:
import json
df['genres_literal'] = df['genres'].apply(json.loads).apply(lambda x : ' '.join([g['name'] for g in x]))

In [98]:
movies_df.head()

,id,title,genres,vote_count,vote_average,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",11800,7.2,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",4500,6.9,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action
2,206647,Spectre,"[Action, Adventure, Crime]",4466,6.3,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Action Adventure Crime
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",9106,7.6,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,Action Crime Drama Thriller
4,49529,John Carter,"[Action, Adventure, Science Fiction]",2124,6.1,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",Action Adventure Science Fiction


In [99]:
# ContVectorizer : 문자열 => vector => sparse matrix
# TFIDF
        tok1, tok2, ..., tok50000
dec1      1    0            2
dec2      0    2            1
dec3      0    2            1

IndentationError: unexpected indent (Temp/ipykernel_1284/718580662.py, line 3)

In [100]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df = 0, ngram_range=(1,2))
genre_mat = vect.fit_transform(movies_df['genres_literal'])

In [101]:
genre_mat.toarray().shape

(4803, 276)

In [102]:
from sklearn.metrics.pairwise import cosine_similarity

In [103]:
genre_sim = cosine_similarity(genre_mat, genre_mat)

In [104]:
genre_sim

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [105]:
sim_idx = genre_sim[0].argsort()[-10:]       # 유사도 높은순서대로 올림차순(오른쪽이 유사도 높은순)

In [106]:
movies_df.loc[sim_idx]

,id,title,genres,vote_count,vote_average,popularity,keywords,overview,genres_literal
420,11253,Hellboy II: The Golden Army,"[Adventure, Fantasy, Science Fiction]",1527,6.5,58.579760,"[auction, northern ireland, resignation, super...",In this continuation to the adventure of the d...,Adventure Fantasy Science Fiction
419,8247,Jumper,"[Adventure, Fantasy, Science Fiction]",1799,5.9,21.218000,"[adolescence, based on novel, loss of child, f...","David Rice is a man who knows no boundaries, a...",Adventure Fantasy Science Fiction
1652,14164,Dragonball Evolution,"[Action, Adventure, Fantasy, Science Fiction, ...",462,2.9,21.677732,"[karate, superhero, revenge, dragon, duringcre...",The young warrior Son Goku sets out on a quest...,Action Adventure Fantasy Science Fiction Thriller
1296,9531,Superman III,"[Comedy, Action, Adventure, Fantasy, Science F...",490,5.3,22.164202,"[saving the world, dc comics, super computer, ...","Aiming to defeat the Man of Steel, wealthy exe...",Comedy Action Adventure Fantasy Science Fiction
14,49521,Man of Steel,"[Action, Adventure, Fantasy, Science Fiction]",6359,6.5,99.398009,"[saving the world, dc comics, superhero, based...",A young boy learns that he has extraordinary p...,Action Adventure Fantasy Science Fiction
46,127585,X-Men: Days of Future Past,"[Action, Adventure, Fantasy, Science Fiction]",6032,7.5,118.078691,"[1970s, mutant, time travel, marvel comic, bas...",The ultimate X-Men ensemble fights a war for t...,Action Adventure Fantasy Science Fiction
870,8536,Superman II,"[Action, Adventure, Fantasy, Science Fiction]",629,6.5,30.515175,"[saving the world, dc comics, sequel, superher...",Three escaped criminals from the planet Krypto...,Action Adventure Fantasy Science Fiction
813,1924,Superman,"[Action, Adventure, Fantasy, Science Fiction]",1022,6.9,48.507081,"[saving the world, journalist, dc comics, crim...",Mild-mannered Clark Kent works as a reporter a...,Action Adventure Fantasy Science Fiction
3494,27549,Beastmaster 2: Through the Portal of Time,"[Action, Adventure, Fantasy, Science Fiction]",17,4.6,1.478505,"[based on novel, time travel, sequel, psychotr...","Mark Singer returns as Dar, the warrior who ca...",Action Adventure Fantasy Science Fiction
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",11800,7.2,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction


In [107]:
pd.DataFrame(genre_sim) # 유사도 metrics

,0,1,2,3,4,5,6,7,8,9,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.596285,0.447214,0.125988,0.755929,0.596285,0.0,0.755929,0.447214,0.745356,...,0.000000,0.000000,0.000000,0.377964,0.000000,0.149071,0.0000,0.000000,0.0,0.0
1,0.596285,1.000000,0.400000,0.169031,0.338062,0.800000,0.0,0.338062,0.600000,0.800000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.0000,0.000000,0.0,0.0
2,0.447214,0.400000,1.000000,0.338062,0.507093,0.600000,0.0,0.507093,0.200000,0.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.0000,0.000000,0.0,0.0
3,0.125988,0.169031,0.338062,1.000000,0.142857,0.169031,0.0,0.142857,0.000000,0.169031,...,0.377964,0.169031,0.377964,0.428571,0.218218,0.676123,0.0000,0.125988,0.0,0.0
4,0.755929,0.338062,0.507093,0.142857,1.000000,0.507093,0.0,1.000000,0.169031,0.507093,...,0.000000,0.000000,0.000000,0.428571,0.000000,0.169031,0.0000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0.149071,0.200000,0.400000,0.676123,0.169031,0.200000,0.0,0.169031,0.000000,0.200000,...,0.000000,0.200000,0.000000,0.169031,0.258199,1.000000,0.0000,0.000000,0.0,0.0
4799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.258199,0.000000,0.000000,0.000000,0.000000,1.0000,0.384900,0.0,0.0
4800,0.000000,0.000000,0.000000,0.125988,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.333333,0.149071,0.333333,0.125988,0.000000,0.000000,0.3849,1.000000,0.0,0.0
4801,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.0


In [108]:
genre_sim.argsort(axis=1)    # 가로방향

array([[2401, 3037, 3038, ...,  813, 3494,    0],
       [2401, 3067, 3069, ...,  129,    1,  262],
       [2401, 2999, 3000, ..., 1542, 1740,    2],
       ...,
       [   0, 2230, 2229, ..., 1895, 3809, 4800],
       [   0, 3205, 3204, ..., 1596, 1594, 4802],
       [   0, 3141, 3140, ..., 4521, 4710, 4802]], dtype=int64)

In [109]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]     # 컬럼 역순
genre_sim_sorted_ind

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]], dtype=int64)

In [110]:
movie_index = 10
movies_df.loc[genre_sim_sorted_ind[movie_index][:5]]  # 상위 10개

,id,title,genres,vote_count,vote_average,popularity,keywords,overview,genres_literal
10,1452,Superman Returns,"[Adventure, Fantasy, Action, Science Fiction]",1400,5.4,57.925623,"[saving the world, dc comics, invulnerability,...",Superman returns to discover his 5-year absenc...,Adventure Fantasy Action Science Fiction
232,76170,The Wolverine,"[Action, Science Fiction, Adventure, Fantasy]",4053,6.3,15.953444,"[japan, samurai, mutant, world war i, marvel c...",Wolverine faces his ultimate nemesis - and tes...,Action Science Fiction Adventure Fantasy
728,1487,Hellboy,"[Fantasy, Action, Science Fiction]",2225,6.5,47.479755,"[black magic, fistfight, cover-up, superhero, ...","In the final days of World War II, the Nazis a...",Fantasy Action Science Fiction
618,9824,Mystery Men,"[Adventure, Fantasy, Action, Comedy, Science F...",250,5.7,12.948627,"[bowling, hostage, sphinx, training, insane as...",When Captain Amazing (Kinnear) is kidnapped by...,Adventure Fantasy Action Comedy Science Fiction
322,18,The Fifth Element,"[Adventure, Fantasy, Action, Thriller, Science...",3885,7.3,109.528572,"[clone, taxi, cyborg, egypt, future, stowaway,...","In 2257, a taxi driver is unintentionally give...",Adventure Fantasy Action Thriller Science Fiction


In [111]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 30):
    
    title_movie = df[df['title'] == title_name]  # 해당 영화 데이터 row
    title_index = title_movie.index.values    # 해당 영화의 index
    
#     print(title_index)
    
    similarity_indexex = sorted_ind[title_index, :(top_n)]  
    # sorted_ind matrix : row - 영화 인덱스 순서, columns - 유사도 순으로 올림차순 영화 인덱스 
    # sorted_ind : 유사도 matrix[해당영화 row index, 왼쪽에서 n개 칼럼]
    
#     print(similarity_indexex)
    
    similarity_indexes = similarity_indexex.reshape(-1)  # 유사도가 높은 영화인덱스들
    
    return df.iloc[similarity_indexes]  # 유사도 높은 영화들의 데이터를 리턴

In [112]:
df = movies_df
sorted_ind = genre_sim_sorted_ind
title_name = 'The Godfather'
top_n = 5

find_sim_movie(df, sorted_ind, title_name, top_n)

,id,title,genres,vote_count,vote_average,popularity,keywords,overview,genres_literal
2731,240,The Godfather: Part II,"[Drama, Crime]",3338,8.3,105.792936,"[italo-american, cuba, vororte, melancholy, pr...",In the continuing saga of the Corleone crime f...,Drama Crime
1243,203,Mean Streets,"[Drama, Crime]",345,7.2,17.002096,"[epilepsy, protection money, secret love, mone...","A small-time hood must choose from among love,...",Drama Crime
3636,36351,Light Sleeper,"[Drama, Crime]",15,5.7,6.063868,"[suicide, drug dealer, redemption, addict, exi...",A drug dealer with upscale clientele is having...,Drama Crime
1946,11699,The Bad Lieutenant: Port of Call - New Orleans,"[Drama, Crime]",326,6.0,17.339852,"[police brutality, organized crime, policeman,...","Terrence McDonagh, a New Orleans Police sergea...",Drama Crime
2640,400,Things to Do in Denver When You're Dead,"[Drama, Crime]",85,6.7,6.932221,"[father son relationship, bounty hunter, boat,...",A mafia film in Tarantino style with a star-st...,Drama Crime


In [113]:
movies_df

,id,title,genres,vote_count,vote_average,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",11800,7.2,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",4500,6.9,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action
2,206647,Spectre,"[Action, Adventure, Crime]",4466,6.3,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Action Adventure Crime
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",9106,7.6,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,Action Crime Drama Thriller
4,49529,John Carter,"[Action, Adventure, Science Fiction]",2124,6.1,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",Action Adventure Science Fiction
...,...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[Action, Crime, Thriller]",238,6.6,14.269792,"[united states–mexico barrier, legs, arms, pap...",El Mariachi just wants to play his guitar and ...,Action Crime Thriller
4799,72766,Newlyweds,"[Comedy, Romance]",5,5.9,0.642552,[],A newlywed couple's honeymoon is upended by th...,Comedy Romance
4800,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie]",6,7.0,1.444476,"[date, love at first sight, narration, investi...","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TV Movie
4801,126186,Shanghai Calling,[],7,5.7,0.857008,[],When ambitious New York attorney Sam is sent t...,


In [114]:
def weighted_vote_average(record):
    v = record['vote_count']
    r = record['vote_average']
    
    record = record[(v > record['vote_count'].mean()) & (r > record['vote_average'].mean())]
    
    record = record.sort_values('popularity',ascending=False)
    
    return record

In [115]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    
    title_movie = df[df['title'] == title_name]  # 해당 영화 데이터 row
    title_index = title_movie.index.values    # 해당 영화의 index
    
#     print(title_index)
    
    similarity_indexex = sorted_ind[title_index, :(top_n)*3]  
    # sorted_ind matrix : row - 영화 인덱스 순서, columns - 유사도 순으로 올림차순 영화 인덱스 
    # sorted_ind : 유사도 matrix[해당영화 row index, 왼쪽에서 n개 칼럼]
    
#     print(similarity_indexex)
    
    similarity_indexes = similarity_indexex.reshape(-1)  # 유사도가 높은 영화인덱스들
    
    record = df.iloc[similarity_indexes]

    final_df = weighted_vote_average(record)
    
    return final_df

In [116]:
df = movies_df
sorted_ind = genre_sim_sorted_ind
title_name = 'The Godfather'
top_n = 5

find_sim_movie(df, sorted_ind, title_name, top_n = 10)

,id,title,genres,vote_count,vote_average,popularity,keywords,overview,genres_literal
3337,238,The Godfather,"[Drama, Crime]",5893,8.4,143.659698,"[italy, love at first sight, loss of father, p...","Spanning the years 1945 to 1955, a chronicle o...",Drama Crime
1881,278,The Shawshank Redemption,"[Drama, Crime]",8205,8.5,136.747729,"[prison, corruption, police brutality, prison ...",Framed in the 1940s for the double murder of h...,Drama Crime
2731,240,The Godfather: Part II,"[Drama, Crime]",3338,8.3,105.792936,"[italo-american, cuba, vororte, melancholy, pr...",In the continuing saga of the Corleone crime f...,Drama Crime
883,640,Catch Me If You Can,"[Drama, Crime]",3795,7.7,73.944049,"[con man, biography, fbi agent, overhead camer...","A true story about Frank Abagnale Jr. who, bef...",Drama Crime
1847,769,GoodFellas,"[Drama, Crime]",3128,8.2,63.654244,"[prison, based on novel, florida, 1970s, mass ...","The true story of Henry Hill, a half-Irish, ha...",Drama Crime
3887,627,Trainspotting,"[Drama, Crime]",2655,7.8,63.513324,"[london england, alcohol, sex, based on novel,...","Renton, deeply immersed in the Edinburgh drug ...",Drama Crime
1149,168672,American Hustle,"[Drama, Crime]",2807,6.8,49.664128,"[con artist, scam, mobster, fbi agent]","A con man, Irving Rosenfeld, along with his se...",Drama Crime
3866,598,City of God,"[Drama, Crime]",1814,8.1,44.356711,"[male nudity, street gang, brazilian, photogra...",Cidade de Deus is a shantytown that started du...,Drama Crime
281,4982,American Gangster,"[Drama, Crime]",1502,7.4,42.361215,"[underdog, black people, drug traffic, drug sm...",Following the death of his employer and mentor...,Drama Crime
2582,97367,The Place Beyond the Pines,"[Drama, Crime]",1653,6.8,35.395129,"[corruption, father son relationship, carnival...",A motorcycle stunt rider considers committing ...,Drama Crime


In [ ]:
import json
df['genres_literal'] = df['genres'].apply(json.loads).apply(lambda x : ' '.join([g['name'] for g in x]))